In [51]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.applications import VGG16

In [27]:
#Labeling Images
def labeling(directory):
    normal_dir = directory + '/NORMAL/'
    pneumonia_dir = directory + '/PNEUMONIA/'
    normal_images = os.listdir(normal_dir)
    pneumonia_images = os.listdir(pneumonia_dir)
    
    # Create a DataFrame with labels
    df = pd.DataFrame({
        "filename": normal_images + pneumonia_images,
        "label": [0] * len(normal_images) + [1] * len(pneumonia_images)  # 0 for normal, 1 for pneumonia
    })
    
    # Shuffle the dataset
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df


In [28]:
train_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
val_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

In [30]:
'''
train_data = labeling(train_path)
test_data = labeling(test_path)
val_data = labeling(val_path)
train_data.head()
'''

'\ntrain_data = labeling(train_path)\ntest_data = labeling(test_path)\nval_data = labeling(val_path)\ntrain_data.head()\n'

In [55]:
ds_train_ = image_dataset_from_directory(
    train_path,
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

ds_test_ = image_dataset_from_directory(
    test_path,
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

ds_val_ = image_dataset_from_directory(
    val_path,
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_test = (
    ds_test_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

ds_val = (
    ds_val_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

model = keras.Sequential([
    VGG16(include_top=False, pooling='avg'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

'''
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=30,
)
'''

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


'\nhistory = model.fit(\n    ds_train,\n    validation_data=ds_val,\n    epochs=30,\n)\n'